# Taiwan Cloropeth Map

In [ ]:
!pip install requests geopandas
!pip install folium

In [ ]:
import folium
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from folium import LayerControl
import os
import base64
from google.colab import drive
drive.mount('/content/drive')

def add_categorical_legend(m, title, colors, labels):
    """
    Add a categorical legend to the map.

    Args:
    - m: folium.Map object
    - title: str, title of the legend
    - colors: list of str, colors for each category
    - labels: list of str, labels for each category
    """
    legend_html = f"""
    <div id="legend" style="position: absolute; bottom: 50px; left: 50px; z-index: 1000;
    background-color: white; border: 2px solid grey; padding: 5px;">
        <h4 style="margin-top: 0;">{title}</h4>
    """
    for color, label in zip(colors, labels):
        legend_html += f"""
        <div style="color: {color}">&#x25A0; {label}</div>
        """
    legend_html += "</div>"
    print("Generated Legend HTML:", legend_html)  # Debugging
    m.get_root().html.add_child(folium.Element(legend_html))
    return m

# Create a map centered on Taiwan
taiwan_map = folium.Map(location=[25.0330, 121.5654], zoom_start=10)

# Add the draggable legend to the map
taiwan_map = add_categorical_legend(taiwan_map, 'Map markers',
                                     colors=['#FF0000', '#008000', '#0000FF'],
                                     labels=['Culture & Historical', 'Natural Wonders', 'Leisure & Entertainment'])

# Define the directory containing the images
#image_dir = "/content/drive/My Drive/places_to_travel"
image_dir = "http://hnglingli.com/Cindy-personal-website/places_to_travel/"


# Define some popular places to travel in Taiwan with their categories
places_to_travel = [
    # Cultural and Historical Sites (Red Markers)
    {
        "name": "Taipei 101",
        "location": [25.0330, 121.5654],
        "image": "taipei_101.jpg",
        "category": "cultural",
        "description": "An architectural marvel symbolizing prosperity, visitors can explore the observatories for stunning city views, making it a must-visit landmark blending tradition and innovation.",
        "map_link": "https://maps.app.goo.gl/5uAUofNZbs71o5gs8"
    },
    {
    "name": "Tainan Anping Fort",
    "location": [22.9943, 120.2033],
    "image": "tainan_anping_fort.jpg",
    "category": "cultural",
    "description": "Anping Fort, a historic fortress built by the Dutch in the 17th century, stands as a testament to Taiwan's rich colonial history. Visitors can explore its well-preserved walls, ancient cannons, and the iconic observation tower for panoramic views of Tainan.",
    "map_link": "https://maps.app.goo.gl/jbKVrUBvxpohnrZe8"
    },
    {
        "name": "Lukang Historic Area",
        "location": [24.0535, 120.4321],
        "image": "lukang_historic_area.jpg",
        "category": "cultural",
        "description": "Lukang Historic Area, known for its intricate wood carvings and traditional architecture, offers a glimpse into Taiwan's vibrant past through its narrow alleys and centuries-old temples.",
        "map_link": "https://maps.app.goo.gl/GbXgctcY3JdsYMnT8"
    },
    {
        "name": "Chiang Kai-shek Memorial Hall",
        "location": [25.0340, 121.5210],
        "image": "chiang_kai_shek_memorial_hall.jpg",
        "category": "cultural",
        "description": "A grand monument in Taipei, commemorates the former President of the Republic of China. The hall features a striking blue octagonal roof and white marble walls, symbolizing abundance and purity. Visitors can explore the main hall with its large statue of Chiang Kai-shek, and enjoy the surrounding gardens and cultural exhibitions.",
        "map_link": "https://maps.app.goo.gl/jboHeZMUy9JKsHSB7"
    },
    {
        "name": "Chimei Museum",
        "location": [23.0308, 120.2651],
        "image": "chimei_museum.jpg",
        "category": "cultural",
        "description": "Features an extensive collection of Western art, musical instruments, and historical artifacts. Established by the Chimei Corporation, the museum is renowned for its European-style architecture and exhibits, including the world's largest violin collection and a diverse array of ancient weapons and animal taxidermy.",
        "map_link": "https://maps.app.goo.gl/qiWmdG1WWhCNs5vT9"
    },
    {
        "name": "Longshan Temple",
        "location": [25.0366, 121.4997],
        "image": "longshan_temple.jpg",
        "category": "cultural",
        "description": "Longshan Temple, built in 1738 by settlers from Fujian, is a prominent religious site in Taipei. Known for its intricate carvings and elegant swallowtail roof, the temple blends Taoist, Buddhist, and Confucian traditions.",
        "map_link": "https://maps.app.goo.gl/sgRWu32AP725nf5f8"
    },
    {
        "name": "Dragon and Tiger Pagodas",
        "location": [22.6247, 120.2672],
        "image": "dragon_and_tiger_pagodas.jpg",
        "category": "cultural",
        "description": "The Dragon and Tiger Pagodas, located at Lotus Pond in Kaohsiung, feature seven-story towers with vibrant dragon and tiger-shaped entrances. Built in 1976, these pagodas are known for their intricate traditional architecture.",
        "map_link": "https://maps.app.goo.gl/mZnZVYyChavFK835A"
    },
    {
        "name": "National Palace Museum",
        "location": [25.1021, 121.5489],
        "image": "national_palace_museum.jpg",
        "category": "cultural",
        "description": "The National Palace Museum in Taipei houses nearly 700,000 pieces of Chinese artifacts and artworks, spanning 8,000 years of history. Known for its vast collection from the imperial collections of the Ming and Qing dynasties, the museum offers a deep dive into China's rich cultural heritage through its meticulously preserved treasures.",
        "map_link": "https://maps.app.goo.gl/1th7KwcMkEKxEDJz7"
    },
    {
        "name": "Ximen Red House",
        "location": [25.0413, 121.5075],
        "image": "ximen_red_house.jpg",
        "category": "cultural",
        "description": "Ximen Red House, built in 1908 during Japanese rule, is a historic theater in Taipei's Ximending district. Known for its distinctive red-brick octagonal structure, it originally served as a market building. Today, it hosts cultural events, performances, and creative markets.",
        "map_link": "https://maps.app.goo.gl/pwECSbM4H653pegr6"
    },


    # Natural Wonders (Green Markers)
    {
        "name": "Taroko National Park",
        "location": [24.1590, 121.6247],
        "image": "taroko_national_park.jpg",
        "category": "natural",
        "description": "Famed for its stunning marble gorge carved by the Liwu River. Visitors can explore dramatic cliffs, lush forests, and the scenic Shakadang Trail, experiencing the park’s unique geological and natural beauty.",
        "map_link": "https://maps.app.goo.gl/fpkaHJuWfLFpzoV97"
    },
    {
        "name": "Sun Moon Lake",
        "location": [23.8634, 120.9292],
        "image": "sun_moon_lake.jpg",
        "category": "natural",
        "description": "Taiwan's largest alpine lake, is renowned for its beauty and cultural significance. Located in Nantou County, the lake is surrounded by lush mountains and is home to the indigenous Thao tribe. Visitors can enjoy scenic boat rides, explore the picturesque Lalu Island, and experience the area's rich cultural heritage.",
        "map_link": "https://maps.app.goo.gl/3Q9DHAcuWbYNtMwHA"
    },
    {
        "name": "Yushan National Park",
        "location": [23.4709, 120.9536],
        "image": "yushan_national_park.jpg",
        "category": "natural",
        "description": "Located in central Taiwan, is home to the island's highest peak, Yushan (Jade Mountain), standing at 3,952 meters. The park features diverse ecosystems, dramatic cliffs, and pristine alpine lakes.",
        "map_link": "https://maps.app.goo.gl/qYqbWaM9qJJNxSv7A"
    },
    {
        "name": "Alishan",
        "location": [23.5084, 120.6059],
        "image": "alishan.jpg",
        "category": "natural",
        "description": "Alishan is renowned for its misty forests, ancient cypress trees, and breathtaking sunrises over a sea of clouds. Visitors can explore the Alishan Forest Railway, hike through scenic trails, and experience the rich culture of the indigenous Tsou tribe.",
        "map_link": "https://maps.app.goo.gl/zovHXPVhAiMcmq9q7"
    },
    {
        "name": "Hualien",
        "location": [23.9872, 121.6015],
        "image": "hualien.jpg",
        "category": "natural",
        "description": "Located on Taiwan's east coast, is known for its stunning natural landscapes, including the breathtaking Taroko Gorge. Visitors can enjoy scenic coastal views, explore lush forests, and experience the vibrant local culture in Hualien City.",
        "map_link": "https://maps.app.goo.gl/icGhYwaVKoWyuv2v9"
    },
    {
        "name": "Shifen Waterfall",
        "location": [25.0655, 121.7359],
        "image": "shifen_waterfall.jpg",
        "category": "natural",
        "description": "A breathtaking natural wonder known as the Little Niagara of Taiwan. Surrounded by lush greenery, offers stunning views and a serene atmosphere.",
        "map_link": "https://maps.app.goo.gl/BVfCfWkyS3hEeL5H9"
    },
    {
        "name": "Kenting National Park",
        "location": [21.9497, 120.7800],
        "image": "kenting_national_park.jpg",
        "category": "natural",
        "description": "Located on the Hengchun Peninsula in Pingtung County. Established in 1984, it features diverse landscapes including tropical beaches, coral reefs, and lush forests. Visitors can explore scenic trails and visit the historic Eluanbi Lighthouse.",
        "map_link": "https://maps.app.goo.gl/Ke4wN9dadcBpCUP96"
    },
    {
        "name": "Yangmingshan National Park",
        "location": [25.1900, 121.5719],
        "image": "yangmingshan_national_park.jpg",
        "category": "natural",
        "description": "Known for its volcanic geography and hot springs, it boasts lush landscapes, vibrant flower fields, and serene hiking trails. Visitors can explore sulfur fumaroles, witness the stunning Qixing Mountain, and enjoy panoramic views of Taipei.",
        "map_link": "https://maps.app.goo.gl/bUkzncNDjQe21NZH7"
    },
    {
        "name": "Penghu Islands",
        "location": [23.5743, 119.5797],
        "image": "penghu_island.jpg",
        "category": "natural",
        "description": "An archipelago of 90 islands in the Taiwan Strait, are known for their stunning beaches, historic sites, and vibrant marine life. Visitors can explore the picturesque Magong City, ancient temples, and enjoy water activities like snorkeling and windsurfing in the crystal-clear waters.",
        "map_link": "https://maps.app.goo.gl/i9kMtSdKXyMawRe69"
    },

    # Leisure and Entertainment (Blue Markers)
    {
        "name": "Taichung Fengjia Night Market",
        "location": [24.1785, 120.6482],
        "image": "taichung_fengjia_night_market.jpg",
        "category": "leisure",
        "description": "One of Taiwan's largest and most famous night markets located near Feng Chia University in Taichung.",
        "map_link": "https://maps.app.goo.gl/73HKfJuc7525hrPY7"
    },
    {
        "name": "Cingjing Farm",
        "location": [24.0297, 121.1755],
        "image": "cingjing_farm.jpg",
        "category": "leisure",
        "description": "Located in Nantou County at an elevation of 1,750 meters, is known for its picturesque mountain scenery and European-style architecture. Visitors can enjoy the Green Green Grasslands, interact with grazing sheep, and explore the Small Swiss Garden, all while experiencing the fresh mountain air and stunning vistas.",
        "map_link": "https://maps.app.goo.gl/Ymntu8c7Y98gh3AB9"
    },
    {
        "name": "Green Island",
        "location": [22.6639, 121.4750],
        "image": "green_island.jpg",
        "category": "leisure",
        "description": "A small volcanic island off the eastern coast of Taiwan. Known for its stunning natural scenery, it features unique geological formations, hot springs, and vibrant coral reefs. Visitors can explore the island's rich marine life, and hike through lush landscapes.",
        "map_link": "https://maps.app.goo.gl/Ef1gFYb54EBqGqnf7"
    },
    {
        "name": "Jiufen Old Street",
        "location": [25.1094, 121.8456],
        "image": "jiufen_old_street.jpg",
        "category": "leisure",
        "description": "Nestled in the mountains of New Taipei City, is famous for its narrow alleyways, traditional teahouses, and stunning views of the Pacific Ocean. This former gold mining town offers a nostalgic atmosphere with its red lanterns, local snacks, and artisan shops, reminiscent of Taiwan's past.",
        "map_link": "https://maps.app.goo.gl/zYvoTrsVMRAUZ6gT6"
    },
    {
        "name": "LIHPAO Discovery Land",
        "location": [24.2572, 120.6464],
        "image": "lihpao_discovery_land.jpg",
        "category": "leisure",
        "description": "Located in Taichung's Houli District, is a premier theme park featuring the world's first tilt roller coaster, Gravity Max. Opened in 1998, the park offers diverse attractions including the Sky Dream Ferris wheel, Mala Bay water park, and various themed areas, providing a fun-filled experience for all ages.",
        "map_link": "https://maps.app.goo.gl/iKi2rYgZs9k6hkm6A"
    },
    {
        "name": "Leofoo Village Theme Park",
        "location": [24.8336, 121.1525],
        "image": "leofoo_village_theme_park.jpg",
        "category": "leisure",
        "description": "Opened in 1994, it features themed areas like Wild West, South Pacific, Arabian Kingdom, and African Safari. Visitors can enjoy thrilling roller coasters, animal encounters, and vibrant live shows, making it a unique entertainment destination.",
        "map_link": "https://maps.app.goo.gl/Ut2JyN9PoFceiwCt6"
    },
    {
        "name": "Janfusun Fancyworld",
        "location": [23.5314, 120.4971],
        "image": "janfusun_fancyworld.jpg",
        "category": "leisure",
        "description": "A popular amusement park that features thrilling rides, including the 88-meter tall Sky Wheel and the Diving Machine G5 roller coaster.",
        "map_link": "https://maps.app.goo.gl/Q13kR5QZ51UJkt9GA"
    },
    {
        "name": "Formosan Aboriginal Culture Village",
        "location": [24.1667, 121.2809],
        "image": "formosan_aboriginal_culture_village.jpg",
        "category": "leisure",
        "description": "This cultural theme park showcases Taiwan's indigenous cultures through performances, exhibits, and traditional architecture.",
        "map_link": "https://maps.app.goo.gl/4weYQfzHET1LhL6n6"
    },
    {
        "name": "E-DA Theme Park",
        "location": [22.6700, 120.3675],
        "image": "e_da_theme_park.jpg",
        "category": "leisure",
        "description": "Located in Kaohsiung's Dashu District, is Taiwan's largest theme park in the south. Opened in 2010, it features Greek-themed architecture and offers over 50 rides and attractions, including the country's largest Ferris wheel and a unique VR experience. The park also includes a large outlet mall and various entertainment shows.",
        "map_link": "https://maps.app.goo.gl/rb9Fzje5MyM4tdWMA"
    },
    {
        "name": "Farglory Ocean Park",
        "location": [22.0154, 120.7514],
        "image": "farglory_ocean_park.jpg",
        "category": "leisure",
        "description": "Taiwan's first marine-themed amusement park that offers a variety of attractions, including water rides, animal exhibits, and educational programs, making it a popular destination for families and tourists alike.",
        "map_link": "https://maps.app.goo.gl/C58vJLsPY8RRdUwk8"
    },
    {
        "name": "Keelung Night Market",
        "location": [25.1294, 121.7421],
        "image": "keelung_night_market.jpg",
        "category": "leisure",
        "description": "Also known as Miaokou Night Market, offers a vibrant atmosphere with hundreds of food stalls serving delicious Taiwanese street food. The market is known for its photogenic rows of yellow lanterns and the historic Dianji Temple at its core.",
        "map_link": "https://maps.app.goo.gl/xtQrKcCyAahC7jgN7"
    },
    {
        "name": "Shilin Night Market",
        "location": [25.0870, 121.5240],
        "image": "shilin_night_market.jpg",
        "category": "leisure",
        "description": "One of Taiwan's largest and most famous night markets. Established in 1909, it features a vast array of food stalls, shops, and entertainment options. Visitors can savor local delicacies like oyster omelettes and stinky tofu, and go shopping for clothes and souvenirs at this bustling market.",
        "map_link": "https://maps.app.goo.gl/kqhkauFoGL25Ntxd9"
    },
    {
        "name": "Raohe Night Market",
        "location": [25.0501, 121.5774],
        "image": "raohe_night_market.jpg",
        "category": "leisure",
        "description": "A vibrant and bustling night market that offers a unique and authentic Taiwanese street food experience.",
        "map_link": "https://maps.app.goo.gl/q6QeLUd6sCMePuuG7"
    },
    {
        "name": "Ningxia Night Market",
        "location": [25.0560, 121.5150],
        "image": "ningxia_night_market.jpg",
        "category": "leisure",
        "description": "Located on Ningxia Road, it's known for its delicious street food, including famous oyster omelets and taro balls.",
        "map_link": "https://maps.app.goo.gl/MwRnnJKBptCCuG8w8"
    },
    {
        "name": "Linjiang Street Night Market",
        "location": [25.0367, 121.5065],
        "image": "linjiang_street_night_market.jpg",
        "category": "leisure",
        "description": "Also known as Tonghua Night Market, it is one of Taipei's top night markets, offering a variety of traditional Taiwanese street foods, including oyster omelettes, barbecued sausages, and ganban noodles.",
        "map_link": "https://maps.app.goo.gl/VQViHietos9EkXr19"
    },
    {
        "name": "Huaxi Night Market",
        "location": [25.0323, 121.5004],
        "image": "huaxi_night_market.jpg",
        "category": "leisure",
        "description": "Known as Snake Alley, it is one of Taipei's oldest and most famous night markets, known for its unique and sometimes controversial offerings, including snake meat and turtle meat.",
        "map_link": "https://maps.app.goo.gl/SrrKJVDhbNTuUWBs6"
    },
    {
        "name": "Zhongli Night Market",
        "location": [24.9653, 121.2214],
        "image": "zhongli_night_market.jpg",
        "category": "leisure",
        "description": "A night market with a vibrant atmosphere, offering a variety of food stalls, local Taiwanese delicacies, games, and shopping options.",
        "map_link": "https://maps.app.goo.gl/DqBEBT9nocgd8yqe8"
    },
    {
        "name": "Luodong Night Market",
        "location": [24.6750, 121.7734],
        "image": "luodong_night_market.jpg",
        "category": "leisure",
        "description": "The largest night market in Yilan County, known for its vibrant local atmosphere and diverse street food. Located in Luodong Township, it features unique Yilan specialties like angelica mutton soup and green onion pancakes.",
        "map_link": "https://maps.app.goo.gl/XUWAxc8GjUh3A6XKA"
    },
    {
        "name": "Yizhong Street Night Market",
        "location": [24.1508, 120.6833],
        "image": "yizhong_street_night_market.jpg",
        "category": "leisure",
        "description": "Located near Chungyo Department Store, it attracts a large crowd, especially students. A great spot to enjoy local culture and a night out with friends.",
        "map_link": "https://maps.app.goo.gl/GCXYxJRvCrTXU7gC7"
    },
    {
        "name": "Wenhua Road Night Market",
        "location": [24.1490, 120.6846],
        "image": "wenhua_road_night_market.jpg",
        "category": "leisure",
        "description": "A bustling night market stretching for about half a kilometer, it transforms into a vibrant pedestrian zone at night, filled with food stalls, shops, and entertainment.",
        "map_link": "https://maps.app.goo.gl/8Fj3xGjFAywvPTF18"
    },
    {
        "name": "Garden Night Market",
        "location": [24.1812, 120.6440],
        "image": "garden_night_market.jpg",
        "category": "leisure",
        "description": "Located in Tainan's North District, it is one of the largest and most popular night markets in Taiwan, known for its wide variety of street food, games, and shopping stalls.",
        "map_link": "https://maps.app.goo.gl/MXHga5Jtsc9RjncB6"
    },
    {
        "name": "Ruifeng Night Market",
        "location": [22.6396, 120.3016],
        "image": "ruifeng_night_market.jpg",
        "category": "leisure",
        "description": "The largest and most vibrant night market in Kaohsiung, located near the Kaohsiung Arena MRT station, featuring over 1,000 stalls offering traditional Taiwanese street foods.",
        "map_link": "https://maps.app.goo.gl/XBKFZ2QfgcdPHdLL7"
    },
    {
        "name": "Liuhe Night Market",
        "location": [22.6252, 120.3022],
        "image": "liuhe_night_market.jpg",
        "category": "leisure",
        "description": "Established in the 1950s, it offers a vibrant atmosphere with around 200 stalls selling a variety of traditional Taiwanese street foods, including fried chicken, squid, shrimp, and crab.",
        "map_link": "https://maps.app.goo.gl/fxwyC6Dmfi32eiat6"
    },
    {
        "name": "Kenting Night Market",
        "location": [21.9481, 120.7894],
        "image": "kenting_night_market.jpg",
        "category": "leisure",
        "description": "A unique night market inside a national park, offering a lively atmosphere with food stalls, game stands, and bars. Enjoy beachside snacks, cocktails, and even fried Oreos while strolling.",
        "map_link": "https://maps.app.goo.gl/W67zA4SRkY6HBPWi7"
    },
]

# Function to determine the marker color based on the category
def get_marker_color(category):
    if category == 'cultural':
        return 'red'
    elif category == 'natural':
        return 'green'
    elif category == 'leisure':
        return 'blue'
    else:
        return 'gray'  # Default color

# Iterate through each place in places_to_travel
for place in places_to_travel:
    # Extract information for each place
    name = place['name']
    location = place['location']
    image_file = place['image']
    category = place['category']
    description = place.get('description', 'No description available.')

    # Construct the full path to the image file
    image_path = os.path.join(image_dir, image_file)


    # Create HTML for the popup with embedded image
    html = f"""
    <h3>{name}</h3>
    <img src="http://hnglingli.com/Cindy-personal-website/places_to_travel/{image_file}" alt="{name}" width="200"><br>
    <p>{description}</p>
    <a href="{place['map_link']}" target="_blank">
        <button>Open in Google Maps</button>
    </a>
    """

    # Create a Popup object with the HTML content
    popup = folium.Popup(html, max_width=300)

    # Determine the marker color based on the category
    marker_color = get_marker_color(category)

    # Add a marker for the place with a popup containing the image
    folium.Marker(
        location=location,
        popup=popup,
        tooltip=name,
        icon=folium.Icon(color=marker_color)
    ).add_to(taiwan_map)

# Define major cities/regions in Taiwan with their coordinates
locations = [
    {'city': 'Taipei', 'lat': 25.0330, 'lon': 121.5654},
    {'city': 'Kaohsiung', 'lat': 22.6273, 'lon': 120.3014},
    {'city': 'Taichung', 'lat': 24.1477, 'lon': 120.6736},
    {'city': 'Tainan', 'lat': 22.9999, 'lon': 120.2270},
    {'city': 'Hualien', 'lat': 23.9872, 'lon': 121.6015},
    {'city': 'Yilan', 'lat': 24.7570, 'lon': 121.7539},
    {'city': 'Chiayi', 'lat': 23.4801, 'lon': 120.4491},
    {'city': 'Pingtung', 'lat': 22.6730, 'lon': 120.4880},
    {'city': 'Taoyuan', 'lat': 24.9934, 'lon': 121.2968},
    {'city': 'Miaoli', 'lat': 24.5600, 'lon': 120.8236},
    {'city': 'Xinbei', 'lat': 25.0172, 'lon': 121.4626},
    {'city': 'Nantou', 'lat': 23.8371, 'lon': 120.9675},
    {'city': 'Taitung', 'lat': 22.7580, 'lon': 121.1434},
    {'city': 'Hsinchu', 'lat': 24.8130, 'lon': 120.9650},
    {'city': 'Hsinchu County', 'lat': 24.7601, 'lon': 121.0092},
    {'city': 'Kenting', 'lat': 21.9630, 'lon': 120.7430},
    {'city': 'Keelung', 'lat': 25.1272, 'lon': 121.7415},
    {'city': 'Changhua', 'lat': 24.0772, 'lon': 120.5378},
    {'city': 'Yunlin', 'lat': 23.7061, 'lon': 120.5429},
    {'city': 'Penghu', 'lat': 23.5664, 'lon': 119.5780},
    {'city': 'Kinmen', 'lat': 24.4370, 'lon': 118.3184},
    {'city': 'Green Island', 'lat': 22.6671, 'lon': 121.4725},
    {'city': 'Lanyu', 'lat': 22.0577, 'lon': 121.5474},
]

buffer_sizes = {
    'Taipei': 0.1,
    'Kaohsiung': 0.14,
    'Taichung': 0.12,
    'Tainan': 0.12,
    'Hualien': 0.16,
    'Yilan': 0.16,
    'Chiayi': 0.12,
    'Pingtung': 0.16,
    'Taoyuan': 0.15,
    'Miaoli': 0.13,
    'Xinbei': 0.15,
    'Nantou': 0.14,
    'Taitung': 0.16,
    'Hsinchu': 0.13,
    'Hsinchu County': 0.14,
    'Kenting': 0.15,
    'Keelung': 0.13,
    'Changhua': 0.14,
    'Yunlin': 0.14,
    'Penghu': 0.16,
    'Kinmen': 0.15,
    'Green Island': 0.05,
    'Lanyu': 0.08,
}

# Your OpenWeatherMap API key
API_KEY = 'ef628c71efe658c36248d8f66b58be86'

# Fetch air quality data
air_quality_data = []

for loc in locations:
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={loc["lat"]}&lon={loc["lon"]}&appid={API_KEY}'
    response = requests.get(url).json()
    aqi = response['list'][0]['main']['aqi']
    air_quality_data.append({
        'city': loc['city'],
        'lat': loc['lat'],
        'lon': loc['lon'],
        'aqi': aqi
    })

# Create DataFrame
air_quality_df = pd.DataFrame(air_quality_data)

# Create GeoDataFrame
geometry = [Point(xy) for xy in zip(air_quality_df['lon'], air_quality_df['lat'])]
geo_df = gpd.GeoDataFrame(air_quality_df, geometry=geometry)

# Create circular polygons around each point with the new buffer sizes
geo_df['geometry'] = geo_df.apply(lambda x: x['geometry'].buffer(buffer_sizes[x['city']]), axis=1)

# Convert to GeoJSON
air_quality_geojson = geo_df.to_json()

# Define color scale for AQI
def get_aqi_color(aqi):
    if aqi == 1:
        return '#009966'  # Good
    elif aqi == 2:
        return '#ffde33'  # Fair
    elif aqi == 3:
        return '#ff9933'  # Moderate
    elif aqi == 4:
        return '#cc0033'  # Poor
    elif aqi == 5:
        return '#660099'  # Very Poor
    else:
        return '#7e0023'  # Hazardous

# Add choropleth for air quality
folium.Choropleth(
    geo_data=air_quality_geojson,
    name='Air Quality',
    data=air_quality_df,
    columns=['city', 'aqi'],
    key_on='feature.properties.city',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Air Quality Index (AQI)'
).add_to(taiwan_map)

# Add Circle Markers for detailed info
for idx, row in air_quality_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=10,
        popup=folium.Popup(f"{row['city']}<br>AQI: {row['aqi']}", max_width=200),
        color=get_aqi_color(row['aqi']),
        fill=True,
        fill_color=get_aqi_color(row['aqi']),
        fill_opacity=0.7
    ).add_to(taiwan_map)

# Add layer control to toggle layers
folium.LayerControl().add_to(taiwan_map)

# Display the map
taiwan_map

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Generated Legend HTML: 
    <div id="legend" style="position: absolute; bottom: 50px; left: 50px; z-index: 1000;
    background-color: white; border: 2px solid grey; padding: 5px;">
        <h4 style="margin-top: 0;">Map markers</h4>
    
        <div style="color: #FF0000">&#x25A0; Culture & Historical</div>
        
        <div style="color: #008000">&#x25A0; Natural Wonders</div>
        
        <div style="color: #0000FF">&#x25A0; Leisure & Entertainment</div>
        </div>


In [ ]:
#save as HTML
taiwan_map.save('taiwan_travel_map.html')